In [0]:
import sympy as sp
import numpy as np
import matplotlib.pyplot as plt

### 1. Formulate the statement of the interpolation problem with Cubic Spline [mathematical formula]

Let y<sub>i</sub> = f(x<sub>i</sub>) to be grid function on $\Omega$<sub>n</sub> and to be defined on [a, b]. We need to approximate it with S<sub>m</sub>(x), where **m** is the function degree.

Given a set of n + 1 data points (x<sub>i</sub>, y<sub>i</sub>) with no two same x<sub>i</sub> and 

\begin{equation*}a=x_{0}<x_{1}<\cdots <x_{n}=b\end{equation*}

the cubic spline S(x) is a function satisfying:

<ul>
  <li> \begin{equation*}S(x)\in C^2[a, b]\end{equation*}</li>
  <li> On each subinterval [x<sub>i</sub>, x<sub>i+1</sub>] S(x) is a polynomial of degree 3, where i = 0, 1, ..., n-1</li>
  </ul>

In [0]:
### YOUR TEXT/CODE HERE










### 2. Formulate the functional and differential compatibility conditions [mathematical formula]

### Functional compatability condition

S(x) is a cubic polynomial, denoted S<sub>i</sub>, on the subinterval [x<sub>i</sub>, x<sub>i+1</sub>] for each i  = 0, 1, ..., n-1:

**S(x<sub>i</sub>) = y<sub>i</sub>**


### Differentail compatability condition

S(x) is a cubic polynomial, denoted S<sub>i</sub>, on the subinterval [x<sub>i</sub>, x<sub>i+1</sub>] for each i  = 0, 1, ..., n-1:

**S''(x<sub>i</sub>) = y''(x<sub>i</sub>)** 


In [0]:
### YOUR TEXT/CODE HERE








### 3. Formulate stitching conditions [mathematical formula]

S(x) is a cubic polynomial, denoted Si, on the subinterval [xi, xi+1] for each i = 0, 1, ..., n-1

<ul>
  
\begin{equation*}S_{m, i+1}^{(p)}(x)|_{x = x_i} = S_{m, i}^{(p)}(x)|_{x = x_i}
  \end{equation*}
  
</ul>

Where p - degree of derrivative, m - spline polynomial degree (in our case, **m == 3**)

In [0]:
### YOUR TEXT/CODE HERE









### 4. Justify why these conditions provide you with the required smoothness [thesis text, no more than 500 characters]

The compatability conditions ensure a cubic spline S(x), S'(x), S''(x) to be continuous on [a, b] . The \begin{equation*}S_{i+1}''(x_{i+1}) = S_{i}(x_{i+1}'')\end{equation*} equation gives spline **smoothness** of degree 2 and ensures the **loss of smoothness** to be 1.

The stitching conditions ensure the derrivatives of degree p to be the same from left and right sides of the x<sub>i</sub>.

These conditions ensure S<sub>3</sub>(x) $\in$ C<sub>2</sub>[a, b] and the existense of continuous derrivatives of 1st and 2nd order.

In [0]:
### YOUR TEXT/CODE HERE










### 5. Derive dependency formula: the dependence of the second derivatives at the grid nodes on the increment of the function (the function values difference on the grid nodes). [Mathematical formulas derivation. Detailed, with clear transitions]

<ul>

<li>\begin{equation*} S_i(x) = S_i(x_i)+s_i'(x_i)(x-x_i)+\frac{s_i''(x_i)}{2}(x-x_i)^2+\frac{s_i'''(x_i)}{6}(x-x_i)^3 = a_i+b_i(x-x_i)+\frac{c_i}{2}(x-x_i)^2+\frac{d_i}{6}(x-x_i)^3]
\end{equation*}</li>

<li>\begin{equation*}a_i = s_i(x_i), b_i = s_i'(x_i), c_i = s_i''(x_i), d_i = s_i'''(x_i)
\end{equation*}</li> by analogy with Taylor series
  
  
 
  <li>Let h<sub>i</sub> = x<sub>i</sub> - x<sub>i-1</sub> - grid step, then using h<sub>i</sub> we van represent a<sub>i-1</sub>, b<sub>i-1</sub>, c<sub>i-1</sub> as
  </li>
  
  <li>\begin{equation*}a_{i-1} = s_{i-1}(x_{i-1}) = s_i(x_{i-1}) = a_i - b_i*h_i+\frac{c_i}{2}h_i^2-\frac{d_i}{h_i^3}, i\in[2..N]
    \end{equation*}
    </li>
  
  <li>\begin{equation*}b_{i-1} =  b_i - c_i*h_i+\frac{d_i}{2}h_i^2, i \in [2, N]
  \end{equation*}</li>
  
  <li>SECOND DERIVATIVE: \begin{equation*}
    c_{i-1} = c_i-d_i*h_i, i \in [2, N]
    \end{equation*}
    </li>
  
  </ul>
  
  
Let denote U(x) as our spline S(x) and u(x) as our original function to avoid misunderstanding in notation.

From the compatability condition:

\begin{equation*}U(x) = s_i(x) = u(x), x\in[x_{i-1}, x_i], i \in [1, N]
\end{equation*} 

\begin{equation*}
a_i = u(x_i)\, from \, substitution\,in\,Q5.2(a_i = s_i(x_i), from\, above\, statement: s_i(x_i) = u(x_i))
\end{equation*}


As our range for a, b, c is from 2..N,
\begin{equation*} for\, x_0:a_1-b_1h_1+\frac{c_1}{2}h_1^2-\frac{d_1}{6}h_1^3 = u(x_0)
\end{equation*}

\begin{cases}a_{i-1} = a_i-b_ih_i+\frac{c_i}{2}h_i^2-\frac{d_i}{6}h_i^3, i\in[2, N]\\b_{i-1} = b_i-c_ih_i+\frac{d_i}{2}h_i^2, i\in [2, N]\\c_{i-1} = c_i-d_ih_i, i \in [2, N]\\a_i = u(x_i), i\in[1, N]\\u(x_0) = a_1-b_1h_1+\frac{c_1}{2}h_1^2-\frac{d_1}{6}h_1^3 \end{cases}



Eliminate all unknown a<sub>i</sub>, i = 1..N using 5th system's equation as u(x<sub>i</sub> - u(x<sub>i-1</sub>)).

\begin{cases}b_ih_i - \frac{c_i}{2}h_i^2+\frac{d_i}{6}h_i^3 = u(x_i) - u(x_{i-1}), i\in[2, N]\\b_{i-1} = b_i-c_ih_i+\frac{d_i}{2}h_i^2, i\in [2, N]\\c_{i-1} = c_i-d_ih_i, i \in [2, N]\\u(x_0) = a_1-b_1h_1+\frac{c_1}{2}h_1^2-\frac{d_1}{6}h_1^3\end{cases}




Eliminate all unknown d<sub>i</sub>:

\begin{cases}b_ih_i - \frac{c_i}{2}h_i^2+\frac{c_i - c_{i-1}}{6}h_i^2 = u(x_i) - u(x_{i-1}), i\in[2, N]\\b_{i-1} = b_i-c_ih_i+\frac{c_i - c_{i-1}}{2}h_i, i\in [2, N]\\u(x_0) = a_1-b_1h_1+\frac{c_1}{2}h_1^2-\frac{d_1}{6}h_1^3 \end{cases}


Divide 1st system's equation by h<sub>i</sub> 


\begin{cases}b_i - \frac{c_i}{2}h_i+\frac{c_i - c_{i-1}}{6}h_i = \frac{u(x_i) - u(x_{i-1})}{h_i}, i\in[2, N]\\b_{i-1} = b_i-c_ih_i+\frac{c_i - c_{i-1}}{2}h_i, i\in [2, N]\\u(x_0) = a_1-b_1h_1+\frac{c_1}{2}h_1^2-\frac{d_1}{6}h_1^3\end{cases}


Group c<sub>i</sub>

\begin{cases}b_i - \frac{c_i}{3}h_i-\frac{c_{i-1}}{6}h_i = \frac{u(x_i) - u(x_{i-1})}{h_i}, i\in[2, N]\\b_{i} - b_{i-1}-\frac{c_ih_i}{2}-\frac{c_{i-1}}{2}h_i = 0, i\in [2, N]\\u(x_0) = a_1-b_1h_1+\frac{c_1}{2}h_1^2-\frac{d_1}{6}h_1^3\end{cases}

Expressing b<sub>i</sub> - b<sub>i-1</sub> via the first equation:

\begin{cases}b_i = \frac{c_ih_i}{3}+\frac{c_{i-1}h_i}{6}+\frac{u(x_i) - u(x_{i-1})}{h_i}, i\in [2, N]\\b_i - b_{i-1} = \frac{c_ih_i}{3}+\frac{c_{i-1}h_i}{6}-\frac{c_{i-1}h_{h-1}}{3}-\frac{c_{i-2}h_{i-1}}{6}+\frac{u(x_i) - u(x_{i-1})}{h_i}-\frac{u(x_{i-2}) - u(x_{i-1})}{h_{i-1}} ,i\in [2, N]\\u(x_0) = a_1-b_1h_1+\frac{c_1}{2}h_1^2-\frac{d_1}{6}h_1^3
\end{cases}

\begin{equation*}\frac{h_{i-1}c_{i-2}}{6}+\frac{h_i+h_{i-1}}{3}c_{i-1}+\frac{h_ic_i}{6} = \frac{u(x_i) - u(x_{i-1})}{h_i}-\frac{u(x_{i-1}) - u(x_{i-2})}{h_{i-1}}, i\in[2, N]
\end{equation*}

or 

\begin{equation*}\frac{h_{i}c_{i-1}}{6}+\frac{h_{i+1}+h_{i}}{3}c_{i}+\frac{h_{i+1}c_{i+1}}{6} = \frac{u(x_{i+1}) - u(x_{i})}{h_{i+1}}-\frac{u(x_{i}) - u(x_{i-1})}{h_{i}}, i\in[1, N-1]
\end{equation*}



In [0]:
### YOUR TEXT/CODE HERE










### 6. Create a system of equations using this formula [Matrix representation. Mathematical formulas]

Assuming even grid step (h<sub>i</sub> = h<sub>i+1</sub>), the system of equation can be rewritten like:

\begin{equation*}\frac{hc_{i-1}}{6}+\frac{2h}{3}c_{i}+\frac{hc_{i+1}}{6} = \frac{u(x_{i+1}) - u(x_{i})}{h{}}-\frac{u(x_{i}) - u(x_{i-1})}{h_{}}, i\in[1, N-1]
\end{equation*}




\begin{vmatrix}\frac{h}{6}&\frac{2h}{3}&\frac{h}{6}&....\\...&\frac{h}{6}&\frac{2h}{3}&\frac{h}{6}\\..&...&...&...\\
\end{vmatrix}* 
\begin{vmatrix}c_0\\..\\.c_n
\end{vmatrix} = 


\begin{vmatrix}\frac{u(x_2)-u(x_1)}{h} - \frac{u(x_1)-u(x_{0})}{h}\\..\\\frac{u(x_n)-u(x_{n-1})}{h} - \frac{u(x_{n-1})-u(x_{n-2})}{h}
\end{vmatrix}

In [0]:
### YOUR TEXT/CODE HERE










### 7. Explain what is an unknown variable in this system. whether the system is closed with respect to an unknown variable. What is missing for closure. [Text, no more than 200 characters]

The unknown variables in the system are c<sub>i</sub> for i across all given grid points. The overall number of unknowns is N, but from our system we can see that we have N-2 equations. Therefore, we can not call the system closed because the solution can not be found in terms of operations from the system (according to definition from http://mathworld.wolfram.com/Closed-FormSolution.html). For system to become closed, we can fullfill it with edge conditions for natural spline - c<sub>0</sub> = c<sub>n</sub> = 0

In [0]:
### YOUR TEXT/CODE HERE










### 8. Bring this matrix to the appropriate form to use the Tridiagonal matrix algorithm [Mathematical derivation. Use Gauss Elimination]

Using edge conditions, \begin{equation*}U''(x_0) = U''(x_N) = 0. 
\end{equation*},  we eliminate $c_0$ and $c_n$ in the system and in the matrix.

Therefore, the matrix will have the following form:

\begin{vmatrix}\frac{2h}{3}&\frac{h}{6}&...&...&...\\\frac{h}{6}&\frac{2h}{3}&\frac{h}{6}&...&...\\...&...&...&...&...\\...&...&\frac{h}{6}&\frac{2h}{3}&\frac{h}{6}\\...&...&...&\frac{h}{6}&\frac{2h}{3}
\end{vmatrix} * 
\begin{vmatrix}c_1\\c_2\\...\\c_{n-1}
\end{vmatrix} = 

\begin{vmatrix}\frac{u(x_3)-u(x_2)}{h} - \frac{u(x_2)-u(x_{1})}{h}\\..\\\frac{u(x_{n-1})-u(x_{n-2})}{h} - \frac{u(x_{n-2})-u(x_{n-3})}{h}
\end{vmatrix}


After applying Gaussian elimination, we will get matrix in the form:

\begin{vmatrix}1&-P1&0&0&...\\0&1&-P2&0&...\\.......\\0&0&0&1&...
\end{vmatrix}*
\begin{vmatrix}c_1\\c_2\\...\\c_{n-1}
\end{vmatrix} = 

\begin{vmatrix}Q1\\..\\Q_l
\end{vmatrix}, 

where $P_j, Q_j\,j\in[1, l]$, l - number of rows in the matrix 
, are coefficients that are found during direct pass of tridiagonal matrix algorithm (they will be found in the next section).


In [0]:
### YOUR TEXT/CODE HERE










### 9. Derive formulas of direct pass and reverse pass of Tridiagonal matrix algorithm [Mathematical formals]

Using explanation and algrithm description from http://scask.ru/i_book_clm.php?id=47, https://pro-prof.com/forums/topic/sweep-method-for-solving-systems-of-linear-algebraic-equations and from Kireev book


## DIRECT PASS

For j = 1..l (l = N-1) (number of rows in the tridiagonal matrix), we need to find coefficents  $P$<sub>j</sub>, $Q$<sub>j</sub> and assuming $\frac{u(x_{i+1}) - u(x_{i})}{h_{i+1}}-\frac{u(x_{i}) - u(x_{i-1})}{h_{i}}$ = $\sigma_j, i\in[1, N-1]$:

**STEP 1**

if j == 1:

  \begin{equation*}
  P_1 = -\frac{1}{4}, Q_1 = \frac{3\sigma_1}{2h}
  \end{equation*}
  
 
 if 1<j<l:

  \begin{equation*}P_j = \frac{\frac{h}{6}}{\frac{-2h}{3}-\frac{h}{6}*P_{j-1}} = -\frac{1}{4} - P_{j-1}
  \end{equation*}, 
  
 $Q_j = \frac{\frac{h}{6}Q_{j-1}-\sigma_j}{\frac{2h}{3} - \frac{h}{6}P_{j-1}}$
  
   if j==l:

  \begin{equation*}Q_l   = \frac{\frac{h}{6}Q_{l-1}-\sigma_l}{\frac{2h}{3} - \frac{h}{6}P_{l-1}}
  \end{equation*}
  
 ## REVERSE PASS
 
 For j = l..1 (l = N-1), we need to find unknowns z<sub>j</sub> (that correspond to our vector of unknowns **c**) by using the formula
 
 <ul>
 <li>\begin{equation*}z_j = P_j*z_{j+1}+Q_{j}
 \end{equation*} for j = k-1..1
 </li>
  
 <li>\begin{equation*}z_l = Q_l
   \end{equation*} for j = k
   </li>
 </ul>
 
In our case:

<ul>
  <li>\begin{equation*} z_l = \frac{r(x_{n-2}, x_{n-1}, x_{n}) - h_{n-1}*\beta_{k-1}}{\gamma_k}
  \end{equation*}</li>
  
  
  <li>\begin{equation*}z_l = Q_l   = \frac{\frac{h}{6}Q_{l-1}-\sigma_l}{\frac{2h}{3} - \frac{h}{6}P_{l-1}}, z_j = (-\frac{1}{4} - P_{j-1})z_{j+1}+\frac{\frac{h}{6}Q_{j-1}-\sigma_j}{\frac{2h}{3} - \frac{h}{6}P_{j-1}}
    \end{equation*}
    </li>
  
  </ul>

In [0]:
### YOUR TEXT/CODE HERE










### 10. Implement code prototype of the future algorithm implementation. Classes/methods (if you use OOP), functions. The final implementation (on language chosen by you) should not differ from the functions declared in the prototype. [Python code]

In [0]:
### Code prototype in Python, but the code will be (probably) implemented in C++

def spline(x, y):
  # x, y - points on the grid and the function value
  n = len(x)-1
  a = y.copy() # coefficicents a for polynomial
  h = [x[i+1]-x[i] for i in range(n)] # for different h
  
  c = [0]*(len(x))
  b = [0]*n
  d = [0]*n
  c[n] = 0
  c[0] = 0
  #h = x[2] - x[1] # for same h 
  
  alpha = [], beta = [], gamma = [] # coefficients for direct pass
  
  alpha.append(...)# add initial coefficients (for same h and different h, the formula
  # is sligtly different)
  gamma.append(...)
  beta.append(...)
  for i in range(1, n):
    
    alpha[i] = #calculating coefficients for direct pass using formulas from
    beta[i] = #https://pro-prof.com/forums/topic/sweep-method-for-solving-systems-of-linear-algebraic-equations
    gamma[i] = #

  gamma.append(...)# append last coefficients
  beta.append(...)
  for i in range(n-1, 0, -1): #reverse pass
    c[i] = .... # calculate c, d, b using formula for reverse pass and then
    b[i] = ....# express d and b from c
    d[i] = ....

    
  return a, b, c, d








### 11. Derive formula of Cubic Spline method error [Mathematical formulas]

Using theorem statement from http://scask.ru/i_book_clm.php?id=89, 
the Cubic Spline approximation error is denoted as
\begin{equation*}max_{a,b}|u(x)^{(p)} - U_3(x)^{(p)}| <=CM_4h_{max}^{4-p}, p = 0, 1, 2.
\end{equation*}, 
where 
<ul>
<li>u(x) - our original function</li> 
  <li>U<sub>3</sub>(x) is the cubic spline</li>
  
  <li>\begin{equation*}M_4 = max_{a,b}|u^{(4)}(x)|
  \end{equation*}</li>, which is the 4th max derrivative of original function
  <li>h<sub>max</sub> - max step between two points in the grid </li>
  
  <li>C - const</li>
  <li>p - degree of derrivative</li>
  </ul>
  
  
 This condition is true if original function u(x) has 4th derrivative. The note here is that the formula approximates derrivatives of degree p too. According to the Theorem explanation from the link, the error assessment is not full for **natural spline** as it has only 2nd accuracy degree, so it leads to lack of accuracy after 2nd derrivative.

In [0]:
### YOUR TEXT/CODE HERE










### 12. Rate the complexity of the algorithm [Text, and rate in terms of big O, no more than 100 characters]

The complexity of the algorithm will be O(n), because:


*   The Tridiagonal matrix algorithm complexity O(n) - direct pass - N, reverse pass - N, N+N = 2N ~ O(N)



In [0]:
### YOUR TEXT/CODE HERE










### Congrats!